## Importação de bibliotecas e Dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install optuna

In [ ]:
%pip install shap

In [ ]:
%pip install imblearn

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import seaborn as sns
import shap
from imblearn.over_sampling import SMOTE
import gdown
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.calibration import calibration_curve
import optuna
import numpy as np
from sklearn.metrics import recall_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
df_premissa = "1Tai4rHDM0P9lGX-kR9gyxAQ80eL8r4SC"
url_fraudes = f"https://drive.google.com/uc?id={df_premissa}"
gdown.download(url_fraudes, quiet=False)

In [ ]:
## https://drive.google.com/file/d/1Tai4rHDM0P9lGX-kR9gyxAQ80eL8r4SC/view?usp=sharing

In [ ]:
dataframe_pj_premissa = pd.read_parquet('/content/pre_processado_pf.parquet')

## Separação de Treino e Teste

In [ ]:
dataframe_pj_premissa.columns.values

In [ ]:
dataframe_pj_premissa = pd.get_dummies(dataframe_pj_premissa, columns=['TIPO_LIGACAO'], dtype=int)

In [ ]:
dataframe_pj_premissa

In [ ]:
# y = dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values
# X = dataframe_pj_premissa.drop(['MATRICULA','DESCRICAO_IRREGULARIDADE IDENTIFICADA', 'DAT_LEITURA', 'MES'], axis=1).values

In [ ]:
for column in dataframe_pj_premissa.columns:
    print(f"Column: {column}, Data type: {dataframe_pj_premissa[column].dtype}")

In [ ]:
# smote = SMOTE(random_state=42)
rus = RandomUnderSampler(random_state=42)
X = dataframe_pj_premissa.drop(['MATRICULA','DESCRICAO_IRREGULARIDADE IDENTIFICADA', 'DAT_LEITURA', 'MES'], axis=1)
y = dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA']
X_res, y_res = rus.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.20, random_state=42)

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import layers

model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(300, activation='relu'),
    layers.Dropout(0.5),
    keras.layers.Dense(300, activation='relu'),
    layers.Dropout(0.5),
    keras.layers.Dense(300, activation='relu'),
    layers.Dropout(0.5),
    keras.layers.Dense(124, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

result = model.fit(X_train, y_train, epochs=60, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Model Accuracy: {accuracy:.2f}')
print(f'Model Precision: {precision:.2f}')
print(f'Model Recall: {recall:.2f}')

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

# Supondo que `result` é o objeto retornado pelo `model.fit`
plot_loss(result)

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Calculando a matriz de confusão em porcentagem
cm_percentage = cm.astype('float') / cm.sum() * 100

# Exibindo a matriz de confusão com porcentagens
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percentage, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)

plt.title('Matriz de Confusão em Porcentagem')
plt.show()

In [ ]:
model_para_timeseares = keras.Sequential(
    [
        layers.Input(shape=(20, 20)),
        layers.Conv1D(
            filters=32,
            kernel_size=7,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1D(
            filters=16,
            kernel_size=7,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Conv1DTranspose(
            filters=16,
            kernel_size=7,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1DTranspose(
            filters=32,
            kernel_size=7,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
    ]
)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

In [ ]:
import pickle

with open("model_PF.pkl", "wb") as file:
    pickle.dump(model, file)

In [ ]:
y_pred_classes = (y_pred > 0.5).astype(int)

print("Classification Report:\n", classification_report(y_test, y_pred_classes))

y_pred_prob = model.predict(X_test).ravel()
roc_auc = roc_auc_score(y_test, y_pred_classes)

# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_classes)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC-ROC (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

### Curva ROC

AUC-ROC (Área Sob a Curva): A área sob a curva ROC é uma métrica que varia entre 0 e 1. Neste caso, o AUC-ROC é 0.70, o que indica que o modelo tem uma habilidade moderada de distinguir entre as classes positivas e negativas. Quanto mais próximo de 1, melhor o desempenho.

Eixo Y (True Positive Rate): Taxa de verdadeiros positivos, ou seja, a proporção de casos positivos corretamente identificados pelo modelo.

Eixo X (False Positive Rate): Taxa de falsos positivos, que indica a proporção de casos negativos incorretamente classificados como positivos.

In [ ]:
plt.plot(result.history['precision'], label='Precisão Treinamento')
plt.plot(result.history['val_precision'], label='Precisão Validação')
plt.title('Curva de Precisão durante o Treinamento')
plt.xlabel('Épocas')
plt.ylabel('Precisão')
plt.legend()
plt.show()

### Curva de Precisão durante o Treinamento

Eixo Y (Precisão): A métrica de precisão para o modelo ao longo do tempo, que é a proporção de previsões corretas feitas em relação a todas as previsões feitas.
Eixo X (Épocas): As épocas são o número de vezes que o modelo percorreu o conjunto de dados durante o treinamento.

Vemos uma melhora progressiva tanto no treinamento quanto na validação, mas com algumas oscilações na validação, o que pode indicar que o modelo está ajustando de maneira instável ao conjunto de validação em certas épocas.

In [ ]:
plt.plot(result.history['loss'], label='Perda Treinamento')
plt.plot(result.history['val_loss'], label='Perda Validação')
plt.title('Curva de Perda durante o Treinamento')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.show()

### Loss

Eixo Y (Perda): Representa a função de perda (error) que o modelo tenta minimizar. No caso de classificação binária, a perda comumente usada é a entropia cruzada.
Eixo X (Épocas): Assim como no gráfico anterior, as épocas representam o progresso do treinamento.

A loss do treinamento e validação diminuem ao longo do tempo, indicando que o modelo está aprendendo. No entanto, há um comportamento instável na loss de validação, sugerindo que o modelo pode estar sofrendo com variações durante as diferentes épocas (possivelmente devido à variação nos dados de validação).

In [ ]:
from sklearn.metrics import precision_recall_curve

y_pred_prob = model.predict(X_test)

precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

plt.plot(recall, precision)
plt.title('Curva de Precisão-Recall')
plt.xlabel('Recall')
plt.ylabel('Precisão')
plt.show()

### Precisão-Recall

A curva precisão-recall é útil em cenários com classes desbalanceadas. A curva decrescente sugere que, à medida que o modelo tenta maximizar o recall (encontrar mais verdadeiros positivos), a precisão diminui, o que é um comportamento esperado.

Eixo Y (Precisão): A proporção de previsões corretas feitas pelo modelo em relação ao número total de previsões.
Eixo X (Recall): Taxa de verdadeiros positivos sobre todos os exemplos positivos. Ou seja, o quão bem o modelo consegue encontrar todas as instâncias positivas.

In [ ]:
prob_true, prob_pred = calibration_curve(y_test, y_pred_prob, n_bins=10)
plt.plot(prob_pred, prob_true, marker='o', label='Calibrated')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect Calibration')
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
plt.title('Calibration Plot')
plt.legend()
plt.show()

## Calibration Plot

Calibration Plot (Gráfico de Calibração), é utilizado para avaliar se as probabilidades previstas por um modelo estão bem calibradas, ou seja, se os valores de probabilidade estimados pelo modelo correspondem à frequência real dos eventos.


O gráfico mostra que a linha azul está próxima da linha de calibração perfeita (linha laranja tracejada), o que sugere que as probabilidades previstas pelo modelo estão bem calibradas em várias regiões, especialmente para previsões acima de 0.6.

Desvio abaixo de 0.4: Para valores de probabilidade menores que 0.4, o modelo tende a ser um pouco menos calibrado, prevendo probabilidades que subestimam a verdadeira taxa de eventos.

Acima de 0.4: A linha azul se alinha bem à linha de calibração perfeita, sugerindo que, para previsões acima de 0.4, o modelo está fornecendo probabilidades bastante precisas.

In [ ]:
from sklearn.metrics import f1_score

thresholds = np.arange(0, 1.1, 0.1)
f1_scores = [f1_score(y_test, (y_pred_prob >= t).astype(int)) for t in thresholds]

plt.figure()
plt.plot(thresholds, f1_scores, marker='o', color='b', label='F1 Score')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Score vs Threshold')
plt.legend(loc='lower right')
plt.show()

### F1 vs Limiar de decisão

O F1 Score varia conforme ajustamos o limiar (threshold) de decisão. O F1 Score é uma métrica que equilibra precisão e recall. Podemos ver que o melhor F1 Score ocorre quando o threshold está em torno de 0.2 a 0.4. Isso indica que, neste intervalo, o modelo consegue o melhor equilíbrio entre precisão e recall. À medida que o threshold aumenta (mais próximo de 1.0), o F1 Score cai drasticamente, sugerindo que o modelo se torna mais conservador, classificando cada vez mais exemplos como negativos, o que diminui o recall, e, consequentemente, o F1 Score.

Ponto de Otimização: O threshold de 0.3 parece ser o ideal, já que o F1 Score está no seu ponto máximo aqui. Ajustar o modelo para trabalhar com esse threshold pode proporcionar uma melhor performance geral.

In [ ]:
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall, precision)

plt.figure()
plt.plot(recall, precision, marker='.', label=f'PR AUC = {pr_auc:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

 ### Precisão vs Recall

 A curva de precisão e recall mostra a relação entre as duas métricas à medida que ajustamos o threshold de decisão. O valor da AUC (Área Sob a Curva) é de 0.79, o que indica que o modelo tem uma boa capacidade de manter um equilíbrio entre precisão e recall. A curva começa alta (próximo de 1.0 tanto em precisão quanto em recall) e diminui à medida que o recall aumenta, o que é esperado.

O valor da AUC de 0.79 é bom, sugerindo que o modelo pode ser eficaz, mas ainda há espaço para melhorias. Para problemas desbalanceados, como o que você provavelmente está enfrentando, essa curva é mais relevante do que a curva ROC. A queda acentuada da precisão para recalls mais altos indica que, ao tentar capturar mais exemplos positivos (aumentando o recall), a precisão diminui significativamente, sugerindo que o modelo está cometendo mais erros ao identificar falsos positivos.

### Acurácia por Cluster

In [ ]:
X_test.columns.values

In [ ]:
np.isnan(X_test).sum()

In [ ]:
print(type(X_test))

In [ ]:
print(X_test.shape)

In [ ]:
X_test_array = X_test.to_numpy()

In [ ]:
matriz = pd.DataFrame(data={'cluster': np.argmax(X_test_array[:,-20:], axis=1),
        'real': y_test,
        'prev': np.argmax(model.predict(X_test_array), axis=1)})

In [ ]:
matriz['acc'] = matriz.real == matriz.prev

In [ ]:
matriz

In [ ]:
matriz[matriz['cluster']==0]

In [ ]:
matriz[matriz['cluster']==0].pivot_table(index='real', columns='prev', values='acc', aggfunc='count')

In [ ]:
acuracia_por_cluster = matriz.pivot_table(index='cluster', values='acc', aggfunc=['sum','count'])

In [ ]:
acuracia_por_cluster['acuracia'] = acuracia_por_cluster[('sum', 'acc')]/acuracia_por_cluster[('count', 'acc')]

In [ ]:
acuracia_por_cluster